# 3.4.0_Dialogue

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import sqlite3

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Building Chatbots in Python'
data = make_dir_line(modality, project)

raw = data('raw')

## 3.4.2 Form filling

In [8]:
# # Define the INIT state
# INIT = 0

# # Define the CHOOSE_COFFEE state
# CHOOSE_COFFEE = 1

# # Define the ORDERED state
# ORDERED = 2

# # Define the policy rules
# policy = {
#     (INIT, "order"): (CHOOSE_COFFEE, "ok, Colombian or Kenyan?"),
#     (INIT, "none"): (INIT, "I'm sorry - I'm not sure how to help you"),
#     (CHOOSE_COFFEE, "specify_coffee"): (ORDERED, "perfect, the beans are on their way!"),
#     (CHOOSE_COFFEE, "none"): (CHOOSE_COFFEE, "I'm sorry - would you like Colombian or Kenyan?"),
# }

# # Create the list of messages
# messages = [
#     "I'd like to become a professional dancer",
#     "well then I'd like to order some coffee",
#     "my favourite animal is a zebra",
#     "kenyan"
# ]

# # Call send_message() for each message
# state = INIT
# for message in messages:    
#     state = send_message(policy, state, message)

USER: I'd like to become a professional dancer
BOT: I'm sorry - I'm not sure how to help you
USER: well then I'd like to order some coffee
BOT: I'm sorry - I'm not sure how to help you
USER: my favourite animal is a zebra
BOT: I'm sorry - I'm not sure how to help you
USER: kenyan
BOT: I'm sorry - I'm not sure how to help you


In [4]:
conn = sqlite3.connect(raw / 'hotels.db')
c = conn.cursor()

In [5]:
df = pd.read_sql_query('SELECT * FROM hotels', conn)
df

,name,price,area,stars
0,Hotel for Dogs,mid,east,3
1,Hotel California,mid,north,3
2,Grand Hotel,hi,south,5
3,Cozy Cottage,lo,south,2
4,Ben's BnB,hi,north,4
5,The Grand,hi,west,5
6,Central Rooms,mid,center,3


In [6]:
c.execute("SELECT * FROM hotels WHERE price = 'mid' AND area = 'north'").fetchall()

[('Hotel California', 'mid', 'north', 3)]

In [7]:
c.close()

## 3.3.3 SQL statements in Python

In [8]:
# Open connection to DB
conn = sqlite3.connect(raw / 'hotels.db')

# Create a cursor
c = conn.cursor()

# Define area and price
area, price = "south", "hi"
t = (area, price)

# Execute the query
c.execute('SELECT * FROM hotels WHERE area=? AND price=?', t).fetchall()

[('Grand Hotel', 'hi', 'south', 5)]

In [9]:
c.close()

## 3.3.5 Creating queries from parameters

In [10]:
# Define find_hotels()
def find_hotels(params):
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0:
        filters = ["{}=?".format(k) for k in params]
        query += " WHERE " + " and ".join(filters)
    # Create the tuple of values
    t = tuple(params.values())
    
    # Open connection to DB
    conn = sqlite3.connect(raw / 'hotels.db')
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

## 3.3.6 Using your custom function to find hotels

In [11]:
# Create the dictionary of column names and values
params = {
    "area":"south",
    "price":"lo"
}

# Find the hotels that match the parameters
print(find_hotels(params))

[('Cozy Cottage', 'lo', 'south', 2)]


## 3.3.7 Creating SQL from natural language

No funciono paquete rasa

In [13]:
# # Define respond()
# def respond(message):
#     # Extract the entities
#     entities = interpreter.parse(message)["entities"]
#     # Initialize an empty params dictionary
#     params = {}
#     # Fill the dictionary with entities
#     for ent in entities:
#         params[ent["entity"]] = str(ent["value"])

#     # Find hotels that match the dictionary
#     results = find_hotels(params)
#     # Get the names of the hotels and index of the response
#     names = [r[0] for r in results]
#     n = min(len(results),3)
#     # Select the nth element of the responses array
#     return responses[n].format(*names)

# # Test the respond() function
# print(respond("I want an expensive hotel in the south of town"))

Respuesta:

    - Grand Hotel is a great hotel!

## 3.3.9 Refining your search

In [ ]:
# # Define a respond function, taking the message and existing params as input
# def respond(message, params):
#     # Extract the entities
#     entities = interpreter.parse(message)["entities"]
#     # Fill the dictionary with entities
#     for ent in entities:
#         params[ent["entity"]] = str(ent["value"])

#     # Find the hotels
#     results = find_hotels(params)
#     names = [r[0] for r in results]
#     n = min(len(results), 3)
#     # Return the appropriate response
#     return responses[n].format(*names), params

# # Initialize params dictionary
# params = {}

# # Pass the messages to the bot
# for message in ["I want an expensive hotel", "in the north of town"]:
#     print("USER: {}".format(message))
#     response, params = respond(message, params)
#     print("BOT: {}".format(response))

Respuesta: 

    USER: I want an expensive hotel
    BOT: Grand Hotel is one option, but I know others too :)
    USER: in the north of town
    BOT: Ben's BnB is a great hotel!

## 3.3.10 Basic negation

In [ ]:
# # Define negated_ents()
# def negated_ents(phrase):
#     # Extract the entities using keyword matching
#     ents = [e for e in ["south", "north"] if e in phrase]
#     # Find the index of the final character of each entity
#     ends = sorted([phrase.index(e) + len(e) for e in ents])
#     # Initialise a list to store sentence chunks
#     chunks = []
#     # Take slices of the sentence up to and including each entitiy
#     start = 0
#     for end in ends:
#         chunks.append(phrase[start:end])
#         start = end
#     result = {}
#     # Iterate over the chunks and look for entities
#     for chunk in chunks:
#         for ent in ents:
#             if ent in chunk:
#                 # If the entity contains a negation, assign the key to be False
#                 if "not" in chunk or "n't" in chunk:
#                     result[ent] = False
#                 else:
#                     result[ent] = True
#     return result  

# # Check that the entities are correctly assigned as True or False
# for test in tests:
#     print(negated_ents(test[0]) == test[1])

Respuesta:

    True
    True
    True
    True

 
## 3.3.11 Filtering with excluded slots

In [ ]:
# Define the respond function
def respond(message, params, neg_params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    ent_vals = [e["value"] for e in entities]
    # Look for negated entities
    negated = negated_ents(message, ent_vals)
    for ent in entities:
        if ent["value"] in negated and negated[ent["value"]]:
            neg_params[ent["entity"]] = str(ent["value"])
        else:
            params[ent["entity"]] = str(ent["value"])
    # Find the hotels
    results = find_hotels(params, neg_params)
    names = [r[0] for r in results]
    n = min(len(results),3)
    # Return the correct response
    return responses[n].format(*names), params, neg_params

# Initialize params and neg_params
params = {}
neg_params = {}

# Pass the messages to the bot
for message in ["I want a cheap hotel", "but not in the north of town"]:
    print("USER: {}".format(message))
    response, params, neg_params = respond(message, params, neg_params)
    print("BOT: {}".format(response))

Respuesta:

    USER: I want a cheap hotel
    BOT: Cozy Cottage is a great hotel!
    USER: but not in the north of town
    BOT: I'm sorry :( I couldn't find anything like that

In [14]:
print('Ok_')

Ok_
